In [1]:
# import all package
import os
import pandas as pd
import snscrape.modules.twitter as scrapper
from datetime import datetime


##### Datat collection and labeling
hapus file df_labeled pada folder store untuk memulai labeling dari awal

In [3]:
# functions scrapper

def scrapByTag(topik, sumIndex, labels):
  TOPIC = topik
  ROW = int(sumIndex)
  name = topik.replace("#", "").replace(' ', '')

  DF_PATH = "".join(['./store/',name, '.csv'])
  LABELED_DF_PATH = "".join(['./store/',name, "_labeled", ".csv"])
  IGNORE_PATH=".gitignore"
  LOG_PATH="log_process.txt"

  TWEETS = []
  ALLOWED_LABELS = labels

  print("DAFTAR INPUT : {}".format("-" * 30))
  print("MASUKAN {} ".format(" atau ".join(ALLOWED_LABELS.keys())))
  for key in ALLOWED_LABELS.keys():
    if(len(key)==1):
      print("{}  = {}".format(key, ALLOWED_LABELS[key]))
    else:
      print("{} = {}".format(key, ALLOWED_LABELS[key]))
  print("SKIP/Skip/skip = untuk melewati tweet")
  print("-" * 30)
  for i, tweet in enumerate(scrapper.TwitterSearchScraper(TOPIC).get_items()):
    if i > ROW:
      break
    label = ""
    while (label not in ALLOWED_LABELS.keys() and label != "SKIP" and label != "Skip" and label != "skip"):
      label = input("{}->({}) =>{}".format(TOPIC,i, tweet.content))

      # label --> hanya mengijinkan label yang terdapat pada dictionary ALLOWED_LABELS
    if(label in ALLOWED_LABELS.keys()):
      TWEETS.append([tweet.date, tweet.id, tweet.user.username,
                    tweet.content, ALLOWED_LABELS[label]])

  dataframe = pd.DataFrame(
      TWEETS, columns=['Datetime', 'Tweet Id', 'Username', 'Text', 'Label'])

# simpan dataframe --> menyimpan dataframe --> ketika script dijalankan dg tagline yang sama maka akan diupdate
  dataframe.to_csv(DF_PATH, index=False)

# simpan df labeled --> df labeled merupakam clone dari dataframe. disimpan pada file berbeda untuk menghindari data tereset (jika tagline yang digunakan sama)
  df_labeled = os.path.exists(LABELED_DF_PATH)
  print("NOTE : {}".format("-" * 30))
  if(not df_labeled):
    dataframe.to_csv(LABELED_DF_PATH, index=False)
    print("-> collection sukses --file {}".format(DF_PATH))
    print("-> labelling sukses --file {}".format(LABELED_DF_PATH))
  else:
    print("-> collection diupdate --file {}".format(DF_PATH))
    print("-> labeling exist --file {}".format(LABELED_DF_PATH))
  print("-" * 30)

# log dan .gitignore --> menacatat waktu collection dan labeling, kemudian mendaftarkan pada .gitignore
  for path in [IGNORE_PATH, LOG_PATH]:
    if(path == IGNORE_PATH):
      with open(path, 'a') as git:
        git.write('\n{}'.format(DF_PATH))
        if(not df_labeled):
          git.write('\n{}'.format(LABELED_DF_PATH))
    elif(path == LOG_PATH):
      with open(path, 'a') as log:
        date = datetime.now().strftime("%d-%b-%Y (%H:%M:%S.%f)")
        log.write("\nlog --collecting {} --on: {} ".format(DF_PATH ,date))
        if(not df_labeled):
          log.write("\nlog --labeling {} --on: {}".format(LABELED_DF_PATH, date))
        log.write("\n----------------------------------------------------------------------------------")

  return dataframe


In [4]:
tag = "#percumaLaporPolisi"
index = 5
labels = {
    "-1": "Negatif",
    "0": "Netral",
    "1": "Positif",
  }
df = scrapByTag(topik=tag, sumIndex=index, labels=labels)
df


DAFTAR INPUT : ------------------------------
MASUKAN -1 atau 0 atau 1 
-1 = Negatif
0  = Netral
1  = Positif
SKIP/Skip/skip = untuk melewati tweet
------------------------------
NOTE : ------------------------------
-> collection diupdate --file ./store/percumaLaporPolisi.csv
-> labeling exist --file ./store/percumaLaporPolisi_labeled.csv
------------------------------


,Datetime,Tweet Id,Username,Text,Label
0,2023-01-06 09:08:05+00:00,1611288484643495937,RutPamela_,Korban jadi tersangka! #percumalaporpolisi #J...,Positif
1,2023-01-06 06:17:55+00:00,1611245658077528065,wuikwok,"@OposisiCerdas Phk biasanya buruh, lah ini phk...",Netral
2,2023-01-05 15:34:02+00:00,1611023224447594496,KelvinWangden1,@txtdrberseragam Apakah #percumalaporpolisi ak...,Negatif
3,2023-01-04 15:25:11+00:00,1610658608634822656,RockButt0m,@secgron #percumalaporpolisi,Positif
